# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)

W = 16
X, Y = rice.shape
bool_rice_without_interpolation = np.zeros((X,Y))
thresholds = np.zeros((X//W,Y//W))
for i in range(0, X, W):
    for j in range (0, Y, W):
        surrounding = rice[i:i+W,j:j+W]
        thresholds[i//W][j//W] = np.mean(surrounding)
for i in range(X):
    for j in range (Y):
        bool_rice_without_interpolation[i][j] = rice[i][j] > thresholds[i//W][j//W]
bool_rice_without_interpolation.astype('int')

fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].imshow(rice,'gray')
ax[0].axis('off')
ax[0].set_title('Obraz')
ax[1].imshow(bool_rice_without_interpolation,'gray')
ax[1].axis('off')
ax[1].set_title('Binaryzacja lokalna w oknach bez interpolacji')
plt.show()

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
def binarization(image, threshold, title =''):
    bool_image = image > threshold
    bool_image.astype('int')
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    ax[0].imshow(image,'gray')
    ax[0].axis('off')
    ax[0].set_title('Obraz')
    ax[1].imshow(bool_image,'gray')
    ax[1].axis('off')
    ax[1].set_title(title)
    plt.show()

def otsu_algorithm(image, title = '', flag = 'd'):
    H = cv2.calcHist([image],[0],None,[256],[0,256])
    H_cum = H.cumsum()
    p = np.squeeze(H/sum(H))
    P = np.squeeze(H_cum/max(H_cum))
    mg = 0
    for i, pi in enumerate(p):
        mg += i*pi
    mg = mg

    s = np.zeros(256)

    for k in range(256):
        m = 0
        for i in range(0, int(k + 1)):
            m += i * p[i]

        s[k] = ((mg * P[k] - m) ** 2) / (P[k] * (1 - P[k]))

    s[np.isnan(s)] = 0
    if flag != 'd':
        plt.plot(s)
        plt.plot()
    threshold = np.argmax(s)
    binarization(image, threshold, title)

def local_algorithm(image, W):
    w = int(W/2)
    X, Y = image.shape
    bool_image = np.zeros((X,Y))
    for j in range(w, Y-w):
        for i in range(w, X-w):
            surrounding = image[i-w:i+w,j-w:j+w]
            mean = np.mean(surrounding)
            bool_image[i][j] = image[i][j] > mean
    bool_image.astype('int')
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    ax[0].imshow(image,'gray')
    ax[0].axis('off')
    ax[0].set_title('Obraz')
    ax[1].imshow(bool_image,'gray')
    ax[1].axis('off')
    ax[1].set_title('Binaryzacja lokalna na podstawie średniaj')
    plt.show()

def sauvoli_pietikainen_algorithm(image, W):
    R = 128
    k = 0.15
    w = int(W/2)
    X, Y = image.shape
    bool_image = np.zeros((X,Y))
    for j in range(w, Y-w):
        for i in range(w, X-w):
            surrounding = image[i-w:i+w,j-w:j+w]
            T = np.mean(surrounding) * (1 + k * ((np.std(surrounding) / R) - 1))
            bool_image[i][j] = image[i][j] > T
    bool_image.astype('int')
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    ax[0].imshow(image,'gray')
    ax[0].axis('off')
    ax[0].set_title('Obraz')
    ax[1].imshow(bool_image,'gray')
    ax[1].axis('off')
    ax[1].set_title('Binaryzacja Sauvoli')
    plt.show()

In [ ]:

bool_rice = np.zeros((X,Y))
rice_thresholds = np.zeros((X, Y))
w = W // 2
for i in range(X):
    for j in range (Y):
        iT, jT = i // W, j // W

        # Rogi
        if (i <= w and j <= w) or (i >= Y - w and j <= w) or (i >= Y - w and j >= X - w) or (i <= w and j >= X - w):
            t11 = thresholds[iT][jT]
            t12 = thresholds[iT][jT]
            t21 = thresholds[iT][jT]
            t22 = thresholds[iT][jT]

        # Poziome
        elif (i <= w or i >= Y - w) and (w <= j <= X - w):
            t11 = thresholds[iT][((j - w) // W)]
            t21 = thresholds[iT][((j - w) // W)]
            t12 = thresholds[iT][1 + (j - w) // W]
            t22 = thresholds[iT][1 + (j - w) // W]

        # Pionowe
        elif (j <= w or j >= X - w) and (w <= i <= Y - w):
            t11 = thresholds[(i - w) // W][jT]
            t21 = thresholds[(i - w) // W][jT]
            t12 = thresholds[1 + (i - w) // W][jT]
            t22 = thresholds[1 + (i - w) // W][jT]

        else:
            t11 = thresholds[iT][(j - w) // W]
            t21 = thresholds[(i - w) // W][jT]
            t12 = thresholds[iT][1 + (j - w) // W]
            t22 = thresholds[1 + (i - w) // W][jT]

        dX1 = j + w - jT * W
        dX2 = (jT+1) * W - j + w
        dY1 = i + w - iT * W
        dY2 = (iT+1) * W - i + w

        rice_thresholds[i][j] = (t11 * (dX2 / W) + t12 * (dX1 / W) + t21 * (dY2 / W) + t22 * (dY1 / W)) / ((dX2 / W) + (dX1 / W) + (dY2 / W) + (dY1 / W))

for i in range(0, X):
    for j in range(0, Y):
        bool_rice[i][j] = rice[i][j] > rice_thresholds[i][j]
bool_rice.astype('int')

fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].imshow(rice,'gray')
ax[0].axis('off')
ax[0].set_title('Obraz')
ax[1].imshow(bool_rice_without_interpolation,'gray')
ax[1].axis('off')
ax[1].set_title('Binaryzacja lokalna w oknach bez interpolacji')
plt.show()

fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].imshow(rice,'gray')
ax[0].axis('off')
ax[0].set_title('Obraz')
ax[1].imshow(bool_rice,'gray')
ax[1].axis('off')
ax[1].set_title('Binaryzacja lokalna w oknach z interpolacją.')
plt.show()

otsu_algorithm(rice, 'Binaryzacja Otsu')

local_algorithm(rice, W)

sauvoli_pietikainen_algorithm(rice, W)

Zdecydowanie binaryzacja adaptacyjna w oknach z interpolacją radzi sobie najlepiej, jeśli chodzi o jednoczesne zachowanie wszystkich szczegółów na zdjęciu, zachowanie niezdeformowanego tła oraz stosunkowo małej liczby artefaktów. W tej samej metodzie, lecz bez interpolacji występuje wiele artefaktów w postaci widocznych granic ramek interpolacji, które interpolacja jest w stanie wyeliminować w dobry sposób. Metoda Otsu radzi sobie lepiej, jeśli chodzi o zachowanie tła i bardzo małą liczbę artefaktów, jednak różnice jasności na zdjęciu powodują niezachowanie przez tą metodę wszystkich szczegółów. Na przykład ziarenko ryżu w prawym dolnym rogu zostało praktycznie usunięte po binaryzacji Otsu, a jego sąsiedzi są zniekształceni. Binaryzacja lokalna na podstawie średniej, co prawda zachowuje dużo szczegółów, lecz tło po binaryzacji ma dużo niedoskonałości. Podobnie jest w przypadku binaryzacji Sauvoli, gdzie tło całkowicie odbiega od oryginału, ale za to poszczególne elementy na zdjęciu są dobrze widoczne, co może być dobrym rozwiązaniem, chociażby podczas identyfikacji obiektów na zdjęciu. Wadą metod binaryzacji lokalnej na podstawie średniej oraz binaryzacji Sauvoli w tej implementacji jest są również powstające czarne ramki na zdjęciu wyjściowym.